In [11]:
!unzip /content/drive/MyDrive/cifar10.tar.gz

Archive:  /content/drive/MyDrive/cifar10.tar.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/cifar10.tar.gz or
        /content/drive/MyDrive/cifar10.tar.gz.zip, and cannot find /content/drive/MyDrive/cifar10.tar.gz.ZIP, period.


In [1]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import cv2

##  Draws bounding boxes on image (numpy array).

In [2]:
def draw_bounding_boxes_on_image_array(image, boxes, color=[], thickness=5):

    # draw_bounding_boxes_on_image(image, boxes, color, thickness)
    boxes_shape = boxes.shape
    if not boxes_shape:
        return
    if len(boxes_shape) != 2 or boxes_shape[1] != 4:
        raise ValueError('Input must be of size [N, 4]')
    for i in range(boxes_shape[0]):
        
        image_width = image.shape[1]
        image_height = image.shape[0]
        cv2.rectangle(image, boxes[i, 1], boxes[i, 0], boxes[i, 3] ,  boxes[i, 2], color[i], thickness)
  
    return image

In [3]:
# Matplotlib config
plt.rc('image', cmap='gray')
plt.rc('grid', linewidth=0)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0')# Matplotlib fonts
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")


# utility to display a row of digits with their predictions
def display_digits_with_boxes(images, pred_bboxes, bboxes, iou, title, bboxes_normalized=False):

    n = len(images)

    fig = plt.figure(figsize=(20, 4))
    plt.title(title)
    plt.yticks([])
    plt.xticks([])
  
    for i in range(n):
      ax = fig.add_subplot(1, 10, i+1)
      bboxes_to_plot = []
      if (len(pred_bboxes) > i):
        bbox = pred_bboxes[i]
        bbox = [bbox[0] * images[i].shape[1], bbox[1] * images[i].shape[0], bbox[2] * images[i].shape[1], bbox[3] * images[i].shape[0]]
        bboxes_to_plot.append(bbox)
    
      if (len(bboxes) > i):
        bbox = bboxes[i]
        if bboxes_normalized == True:
          bbox = [bbox[0] * images[i].shape[1],bbox[1] * images[i].shape[0], bbox[2] * images[i].shape[1], bbox[3] * images[i].shape[0] ]
        bboxes_to_plot.append(bbox)

      img_to_draw = draw_bounding_boxes_on_image_array(image=images[i], boxes=np.asarray(bboxes_to_plot), color=[(255,0,0), (0, 255, 0)])
      plt.xticks([])
      plt.yticks([])
    
      plt.imshow(img_to_draw)

      if len(iou) > i :
        color = "black"
        if (iou[i][0] < iou_threshold):
          color = "red"
        ax.text(0.2, -0.3, "iou: %s" %(iou[i][0]), color=color, transform=ax.transAxes)
        

# read_image_tfds_with_original_bbox:

-This function reads image from data

-It also denormalizes the bounding boxes (it undoes the bounding box normalization that is performed by the previous two helper functions.)

In [4]:
def read_image_tfds_with_original_bbox(data):
    image = data["image"]
    bbox = data["bbox"]

    shape = tf.shape(image)
    factor_x = tf.cast(shape[1], tf.float32) 
    factor_y = tf.cast(shape[0], tf.float32)

    bbox_list = [bbox[1] * factor_x , 
                 bbox[0] * factor_y, 
                 bbox[3] * factor_x, 
                 bbox[2] * factor_y]
    return image, bbox_list

# converts a dataset into numpy arrays of images and boxes 

In [5]:
def dataset_to_numpy_util(dataset, batch_size=0, N=0):


    take_dataset = dataset.shuffle(1024)

    if batch_size > 0:
        take_dataset = take_dataset.batch(batch_size)
  
    if N > 0:
        take_dataset = take_dataset.take(N)
        
  #Checks whether the current thread has eager execution enabled.
    if tf.executing_eagerly():
        ds_images, ds_bboxes = [], []
        for images, bboxes in take_dataset:
            ds_images.append(images.numpy())
            ds_bboxes.append(bboxes.numpy())
        
    return (np.array(ds_images), np.array(ds_bboxes))

## Visualize the training images and their bounding box labels

In [6]:
def get_visualization_training_dataset():      
    #	with_info : bool, if True, tfds.load will return the tuple (tf.data.Dataset, tfds.core.DatasetInfo), the latter containing the info associated with the builder.
    dataset, info = tfds.load("Pet_Breeds", split="train", with_info=True, data_dir=data_dir, download=False)
    print(info)
    visualization_training_dataset = dataset.map(read_image_tfds_with_original_bbox, 
                                                 num_parallel_calls=16)
    
    ds = tf.data.Dataset.from_tensor_slices(dict(data_dir)).\
        map(read_image_tfds_with_original_bbox ,num_parallel_calls=16).\
        batch(4)


    return visualization_training_dataset

visualization_training_dataset = get_visualization_training_dataset()

NameError: ignored

In [ ]:
(visualization_training_images, visualization_training_bboxes) =dataset_to_numpy_util(visualization_training_dataset, N=10)
display_digits_with_boxes(np.array(visualization_training_images), np.array([]), np.array(visualization_training_bboxes), np.array([]), "training images and their bboxes")

In [ ]:
  
dataset = tf.keras.preprocessing.image_dataset_from_directory(
         data_dir,
      
         seed=123,
         image_size=(240, 240),
         batch_size=64)

print(type(dataset))

Found 3881 files belonging to 23 classes.
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


In [ ]:
import tensorflow_datasets as tfds
builder = tfds.builder('Pet_Breeds', data_dir='/content/ds_pet_breeds/Pet_Breeds')
builder.download_and_prepare() # this can take a few minutes
exit

DatasetNotFoundError: ignored